In [0]:
import keras
from keras.models import Sequential, Model, load_model
from keras.layers.core import Dense, Flatten
from keras.applications.vgg16 import VGG16 
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping
import keras.backend as K
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import cv2
import h5py
import matplotlib.pyplot as plt
from google.colab import drive
import pandas as pd

In [0]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [0]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd "drive/My Drive/BTP"

/content/drive/My Drive/BTP


In [0]:
from bateaux_metrics_v2 import *

In [0]:
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'vgg_v2.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

model_path = os.path.join(save_dir, model_name)

In [0]:
from tensorflow.python.client import device_lib
from keras import backend
assert len(backend.tensorflow_backend._get_available_gpus()) > 0
assert 'GPU' in str(device_lib.list_local_devices())

In [0]:
K.clear_session()

In [0]:
img_width = 128
img_height = 128
channels = 3

In [0]:
base_model = VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape=(img_width, img_height, channels))




58892288/58889256 [==============================] - 2s 0us/step




In [0]:
for layer in base_model.layers[:-4]:
  layer.trainable = False
layers = [(layer, layer.name, layer.trainable) for layer in base_model.layers]
layer_info = pd.DataFrame(layers, columns=['Layer type', 'Layer name', 'Layer trainable'])
pd.set_option('display.max_rows', layer_info.shape[0]+1)
layer_info

,Layer type,Layer name,Layer trainable
0,<keras.engine.input_layer.InputLayer object at...,input_1,False
1,<keras.layers.convolutional.Conv2D object at 0...,block1_conv1,False
2,<keras.layers.convolutional.Conv2D object at 0...,block1_conv2,False
3,<keras.layers.pooling.MaxPooling2D object at 0...,block1_pool,False
4,<keras.layers.convolutional.Conv2D object at 0...,block2_conv1,False
5,<keras.layers.convolutional.Conv2D object at 0...,block2_conv2,False
6,<keras.layers.pooling.MaxPooling2D object at 0...,block2_pool,False
7,<keras.layers.convolutional.Conv2D object at 0...,block3_conv1,False
8,<keras.layers.convolutional.Conv2D object at 0...,block3_conv2,False
9,<keras.layers.convolutional.Conv2D object at 0...,block3_conv3,False


In [0]:
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(6))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 49158     
Total params: 14,763,846
Trainable params: 7,128,582
Non-trainable params: 7,635,264
_________________________________________________________________


In [0]:
y_file = 'train_GT.csv'
y_train = np.loadtxt(y_file, dtype=np.float32)

x_file = 'train_set.hdf5'
h5_reader = h5py.File(x_file, 'r')
print(h5_reader.keys())
x_train = h5_reader['train_img']
x_train = np.array(x_train)
h5_reader.close()

KeysView(<HDF5 file "train_set.hdf5" (mode r)>)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=12)

In [0]:
img_gen = ImageDataGenerator()

In [0]:
epochs = 100
batch_size = 32
epochs_to_wait_for_improve = 4
steps_per_epoch = X_train.shape[0] // batch_size
print(steps_per_epoch)

742


In [0]:
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=adam, loss=euclidean_loss(beta=beta), metrics=[euclidean_dist, x_error, y_error, z_error])

In [0]:
early_stopping_callback = EarlyStopping(monitor='euclidean_dist', patience=epochs_to_wait_for_improve, restore_best_weights=True)

In [0]:
model.fit_generator(img_gen.flow(X_train, Y_train, batch_size=batch_size),
            steps_per_epoch=steps_per_epoch, validation_data=(X_val, Y_val),
            epochs=epochs, callbacks=[early_stopping_callback])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
742/742 [==============================] - 76s 103ms/step - loss: 0.8381 - euclidean_dist: 0.0818 - x_error: 0.0481 - y_error: 0.1011 - z_error: 0.0561 - val_loss: 0.4203 - val_euclidean_dist: 0.0275 - val_x_error: 0.0249 - val_y_error: 0.0251 - val_z_error: 0.0246
Epoch 2/100
742/742 [==============================] - 67s 91ms/step - loss: 0.4235 - euclidean_dist: 0.0278 - x_error: 0.0250 - y_error: 0.0251 - z_error: 0.0252 - val_loss: 0.4204 - val_euclidean_dist: 0.0276 - val_x_error: 0.0249 - val_y_error: 0.0254 - val_z_error: 0.0246
Epoch 3/100
742/742 [==============================] - 68s 91ms/step - loss: 0.4237 - euclidean_dist: 0.0278 - x_error: 0.0250 - y_error: 0.0251 - z_error: 0.0251 - val_loss: 0.4202 - val_euclidean_dist: 0.0276 - val_x_error: 0.0250 - val_y_error: 0.0251 - val_z_error: 0.0246
Epoch 4/100
742/742 [==============================] - 68s 91ms/step - l

In [0]:
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at /content/drive/My Drive/BTP/saved_models/vgg_v2.h5 


In [0]:
x_file = 'test_set.hdf5'
h5_reader = h5py.File(x_file, 'r')
#print(h5_reader.keys())
x_test = h5_reader['test_img']
x_test = np.array(x_test)
h5_reader.close()
x_test = x_test.astype('float32')
x_test /= 255

In [0]:
y_file = 'test_GT.csv'
y_test = np.loadtxt(y_file, dtype=np.float32)

In [0]:
metric_names = model.metrics_names
scores = model.evaluate(x_test, y_test, verbose=1)
for metric, val in zip(metric_names, scores):
  print("{} : {}".format(metric, val))

6600/6600 [==============================] - 13s 2ms/step
loss : 0.4188131729039279
euclidean_dist : 0.027791009236014252
x_error : 0.024985134479674424
y_error : 0.025048343205090726
z_error : 0.025129301177732873
